- Author: **Võ Bách Khôi** | 19127037
- - -
Notebook này dùng để thu thập dữ liệu từ **Soundcloud**. Chúng ta sẽ thu thập dữ liệu về:
- User
- Playlist
- Track

Đầu tiên là import các thư viện cần thiết.

In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime

Setup cache trước khi tiến hàng cào dữ liệu :"D

In [2]:
sleep_time = 1
requests_cache.install_cache('http_cache',expire_after=None)

Ta sẽ tiến hành cào dữ liệu của các user đã được chuẩn bị trước.

In [3]:
urls_df = pd.read_csv('urls.csv', index_col=0)
urls_df

,username,url,playlist_sets,playlist
0,lilcandypaint *Candypaint,/lilcandypaint,https://soundcloud.com/lilcandypaint/sets,"/lilcandypaint/sets/its-still-purple,/lilcandy..."
1,Nardo Wick,/nardo-wick,https://soundcloud.com/nardo-wick/sets,NaN
2,Mohamed Ramadan,/mohamedramadanofficial,https://soundcloud.com/mohamedramadanofficial/...,NaN
3,Lil Tjay,/liltjay,https://soundcloud.com/liltjay/sets,/liltjay/sets/no-comparison
4,muhammed fawzii,/muhammed-fawzii,https://soundcloud.com/muhammed-fawzii/sets,"/muhammed-fawzii/sets/music,/muhammed-fawzii/s..."
...,...,...,...,...
2024,MyApple.pl,/myapplepl,https://soundcloud.com/myapplepl/sets,"/myapplepl/sets/myapple-weekly,/myapplepl/sets..."
2025,Tecnocast,/tecnoblog,https://soundcloud.com/tecnoblog/sets,/tecnoblog/sets/tecnoinvest
2026,The Tech Guy,/the_tech_guy,https://soundcloud.com/the_tech_guy/sets,NaN
2027,This Week in Tech,/this-week-in-tech,https://soundcloud.com/this-week-in-tech/sets,NaN


Do **Soundcloud** giới hạn số lần request trong một ngày nên có thể xảy ra lỗi thì chạy hàm này :( Do đó khi có lỗi, ta sẽ trả về những gì đã lấy được

In [4]:
def get_userdata(user_urls_df, user_attributes):
    # Khởi tạo dữ liệu người dùng rỗng
    user_data = []

    for url in user_urls_df:
        # Nếu request xảy ra lỗi, trả về những gì đã lấy được
        try:
            r = requests.get('https://soundcloud.com' + url)
            r.encoding = 'utf-8'
        except:
            return user_data

        # Nếu request không từ cache thì 
        # delay để tránh hit trang web nhiều lần trong thời gian ngắn
        if (r.from_cache == False):
            time.sleep(sleep_time)
        
        # Parse trang web bằng BeautifulSoup
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # Tìm thẻ script chứa "window.__sc_hydration"
        for script in soup.find_all('script'):
            data = script.string
            if data: 
                if 'window.__sc_hydration' in data:
                    data = data[25:-2]
                    idx = data.find('{"hydratable":"user"')
                    if (idx == -1): break
                    data = json.loads(data[idx:])
                    row = [data['data'][att] for att in user_attributes]
                    user_data.append(row)
                    break
        
    return user_data

Chúng ta chỉ lọc ra một số cột quan trọng mà thôi

In [5]:
user_attributes = ['avatar_url', 'city', 'comments_count', 'country_code', 'created_at', 'followers_count', 'followings_count',
                  'first_name', 'id', 'last_modified', 'last_name', 'likes_count', 'playlist_likes_count', 'permalink_url',
                  'playlist_count', 'track_count', 'username', 'verified']
user_data = get_userdata(urls_df['url'], user_attributes)

In [6]:
user_df = pd.DataFrame(data=user_data, columns=user_attributes)
user_df

,avatar_url,city,comments_count,country_code,created_at,followers_count,followings_count,first_name,id,last_modified,last_name,likes_count,playlist_likes_count,permalink_url,playlist_count,track_count,username,verified
0,https://i1.sndcdn.com/avatars-5WZMC6IGwNozrsfT...,chicago$atlanta,3,US,2015-10-08T03:14:51Z,44230,13,candy,179256009,2021-04-22T17:51:28Z,paint,389,14,https://soundcloud.com/lilcandypaint,8,96,lilcandypaint *Candypaint,True
1,https://i1.sndcdn.com/avatars-YUbVq2h4Fhb3PPEg...,jacksonville,0,US,2020-04-18T23:13:35Z,19784,0,Nardo,811818505,2021-03-03T18:11:28Z,Wick,6,0,https://soundcloud.com/nardo-wick,0,11,Nardo Wick,False
2,https://i1.sndcdn.com/avatars-000462235182-pyl...,None,0,EG,2018-06-22T19:31:04Z,246143,0,Mohamed,466991778,2021-06-02T23:49:32Z,Ramadan,0,0,https://soundcloud.com/mohamedramadanofficial,2,51,Mohamed Ramadan,True
3,https://i1.sndcdn.com/avatars-0IvIfvallcx6Rypx...,Bronx,1,None,2016-08-03T12:11:59Z,784131,2,,245152552,2021-08-14T19:02:28Z,,13,0,https://soundcloud.com/liltjay,6,67,Lil Tjay,True
4,https://i1.sndcdn.com/avatars-QWEy2uIWDKkNRt9N...,,6,EG,2014-01-22T16:11:44Z,2791,28,Muhamed,76617335,2021-06-07T00:11:47Z,fawzi,750,17,https://soundcloud.com/muhammed-fawzii,2,7,muhammed fawzii,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,https://i1.sndcdn.com/avatars-000062965281-pvu...,,5,PL,2013-12-15T21:15:22Z,395,1,,70584473,2019-11-08T22:18:39Z,,1,0,https://soundcloud.com/myapplepl,4,617,MyApple.pl,False
2025,https://i1.sndcdn.com/avatars-JO0sSvPsnPeW9Q52...,,44,BR,2012-01-31T19:41:53Z,7908,3,Tecnoblog,11458857,2021-04-29T21:15:54Z,,2,1,https://soundcloud.com/tecnoblog,1,224,Tecnocast,True
2026,https://i1.sndcdn.com/avatars-000163838094-anq...,"Petaluma, CA",0,None,2015-08-19T02:16:46Z,534,1,Leo,169185192,2019-10-01T00:02:26Z,Laporte,0,0,https://soundcloud.com/the_tech_guy,0,631,The Tech Guy,False
2027,https://i1.sndcdn.com/avatars-000164066046-hlt...,"Petaluma, CA",0,US,2015-08-19T19:53:56Z,862,3,,169313522,2019-10-01T00:02:27Z,,0,0,https://soundcloud.com/this-week-in-tech,0,333,This Week in Tech,False


In [7]:
user_df.to_csv('user.csv')

Tiếp tục cào dữ liệu các playlist

In [8]:
urls_df['playlist'].fillna('None', inplace=True)

In [9]:
def get_playlistdata(playlist_urls_df, playlist_attributes):
    # Khởi tạo dữ liệu
    playlist_data = []
    
    # Duyệt qua danh sách các playlist của mỗi user
    for playlists in playlist_urls_df:
        # Nếu user này không có playlist nào
        if playlists == 'None':
            pass
        else:
            # Các urls phân cách nhau bởi dấu ','
            playlist_urls = playlists.split(',')
            # Nếu request xảy ra lỗi, trả về những gì đã lấy được
            for playlist_url in playlist_urls:
                try:
                    r = requests.get('https://soundcloud.com' + playlist_url)
                    r.encoding = 'utf8'
                except:
                    return playlist_data
                
                # Nếu request không từ cache thì 
                # delay để tránh hit trang web nhiều lần trong thời gian ngắn
                if (r.from_cache == False):
                    time.sleep(sleep_time)
                
                # Parse trang web bằng BeautifulSoup
                soup = BeautifulSoup(r.content, 'html.parser')
                
                # Tìm thẻ script chứa "window.__sc_hydration"
                for script in soup.find_all('script'):
                    data = script.string
                    if data: 
                        if 'window.__sc_hydration' in data:
                            data = data[25:-2]
                            idx = data.find('{"hydratable":"playlist"')
                            if (idx == -1): break
                            data = json.loads(data[idx:])
                            row = [data['data'][att] for att in playlist_attributes]
                            playlist_data.append(row)
                            break
                            
    return playlist_data

In [10]:
playlist_attributes = ['artwork_url', 'created_at', 'duration', 'genre', 'id', 'label_name', 'last_modified', 
                       'license','likes_count', 'permalink_url', 'release_date', 'purchase_title', 'purchase_url', 
                       'reposts_count', 'tag_list', 'title', 'user_id', 'is_album', 'published_at', 'display_date', 
                       'track_count', 'tracks']
playlist_data = get_playlistdata(urls_df['playlist'], playlist_attributes)

In [11]:
playlist_df = pd.DataFrame(data=playlist_data, columns=playlist_attributes)
playlist_df

,artwork_url,created_at,duration,genre,id,label_name,last_modified,license,likes_count,permalink_url,...,purchase_url,reposts_count,tag_list,title,user_id,is_album,published_at,display_date,track_count,tracks
0,None,2020-02-18T23:56:36Z,570567,,994786066,None,2020-02-18T23:56:36Z,all-rights-reserved,2331,https://soundcloud.com/lilcandypaint/sets/its-...,...,None,150,,its still purple,179256009,False,2020-02-18T23:56:36Z,2020-02-18T23:56:36Z,4,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
1,None,2018-11-05T23:23:03Z,818642,,638014500,None,2018-11-05T23:23:03Z,all-rights-reserved,1769,https://soundcloud.com/lilcandypaint/sets/s8ta...,...,None,147,,S8TANS LAST DANCE,179256009,False,2018-11-05T23:23:03Z,2018-11-05T23:23:03Z,4,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
2,https://i1.sndcdn.com/artworks-000434498733-nd...,2018-11-05T23:21:56Z,818642,,638013903,None,2018-11-05T23:27:16Z,all-rights-reserved,1078,https://soundcloud.com/lilcandypaint/sets/s8ta...,...,None,83,,S8TANS LAST DANCE,179256009,False,2018-11-05T23:21:56Z,2018-11-05T23:21:56Z,4,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
3,None,2016-08-30T01:31:01Z,105491,INTERNETMUSIC,254786558,None,2016-08-30T01:31:36Z,all-rights-reserved,460,https://soundcloud.com/lilcandypaint/sets/inte...,...,None,52,,INTERNET,179256009,False,2016-08-30T01:31:01Z,2016-08-30T01:31:01Z,1,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
4,None,2018-12-24T22:00:08Z,939810,,670946511,None,2018-12-24T22:09:56Z,all-rights-reserved,21484,https://soundcloud.com/liltjay/sets/no-comparison,...,None,1062,,No Comparison,245152552,False,2018-12-24T22:00:08Z,2018-12-24T22:00:08Z,5,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3760,https://i1.sndcdn.com/artworks-000247209438-zk...,2017-10-14T19:00:50Z,35217852,,359984842,None,2019-01-14T02:14:46Z,all-rights-reserved,1,https://soundcloud.com/myapplepl/sets/myapple-...,...,None,0,,MyApple Weekly,70584473,False,2017-10-14T19:00:50Z,2017-10-14T19:00:50Z,29,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
3761,https://i1.sndcdn.com/artworks-000247209167-pl...,2014-12-09T22:39:17Z,263092108,Technology,63234214,,2017-10-14T18:57:52Z,all-rights-reserved,43,https://soundcloud.com/myapplepl/sets/myapple-...,...,None,2,podcast myapple daily apple iphone ipad mac,MyApple Daily,70584473,False,None,2014-12-09T22:39:17Z,431,"[{'artwork_url': None, 'caption': None, 'comme..."
3762,https://i1.sndcdn.com/artworks-aFz7PdQlvKJMLRu...,2013-12-22T20:16:03Z,973451449,Technology,17908634,None,2021-10-31T16:48:29Z,all-rights-reserved,40,https://soundcloud.com/myapplepl/sets/macgadka,...,None,3,apple mac iphone ipad macos ios,MacGadka 🎙,70584473,False,None,2013-12-22T20:16:03Z,138,"[{'artwork_url': None, 'caption': None, 'comme..."
3763,None,2013-12-16T10:16:59Z,43472259,,17326482,,2013-12-17T20:49:30Z,all-rights-reserved,6,https://soundcloud.com/myapplepl/sets/podcast-...,...,None,1,,Podcast MyApple 2006 Season 01,70584473,False,None,2013-12-16T10:16:59Z,15,"[{'artwork_url': None, 'caption': None, 'comme..."


In [12]:
track_raw_data = playlist_df['tracks'].copy()

In [13]:
track_ids = []
for tracks in playlist_df['tracks']:
    row = [str(track['id']) for track in tracks]
    track_ids.append(','.join(row))
playlist_df['tracks'] = track_ids

In [14]:
playlist_df

,artwork_url,created_at,duration,genre,id,label_name,last_modified,license,likes_count,permalink_url,...,purchase_url,reposts_count,tag_list,title,user_id,is_album,published_at,display_date,track_count,tracks
0,None,2020-02-18T23:56:36Z,570567,,994786066,None,2020-02-18T23:56:36Z,all-rights-reserved,2331,https://soundcloud.com/lilcandypaint/sets/its-...,...,None,150,,its still purple,179256009,False,2020-02-18T23:56:36Z,2020-02-18T23:56:36Z,4,"762943381,762943480,762943408,762943528"
1,None,2018-11-05T23:23:03Z,818642,,638014500,None,2018-11-05T23:23:03Z,all-rights-reserved,1769,https://soundcloud.com/lilcandypaint/sets/s8ta...,...,None,147,,S8TANS LAST DANCE,179256009,False,2018-11-05T23:23:03Z,2018-11-05T23:23:03Z,4,"525364131,525364092,525364122,525364077"
2,https://i1.sndcdn.com/artworks-000434498733-nd...,2018-11-05T23:21:56Z,818642,,638013903,None,2018-11-05T23:27:16Z,all-rights-reserved,1078,https://soundcloud.com/lilcandypaint/sets/s8ta...,...,None,83,,S8TANS LAST DANCE,179256009,False,2018-11-05T23:21:56Z,2018-11-05T23:21:56Z,4,"525364092,525364131,525364122,525364077"
3,None,2016-08-30T01:31:01Z,105491,INTERNETMUSIC,254786558,None,2016-08-30T01:31:36Z,all-rights-reserved,460,https://soundcloud.com/lilcandypaint/sets/inte...,...,None,52,,INTERNET,179256009,False,2016-08-30T01:31:01Z,2016-08-30T01:31:01Z,1,280512672
4,None,2018-12-24T22:00:08Z,939810,,670946511,None,2018-12-24T22:09:56Z,all-rights-reserved,21484,https://soundcloud.com/liltjay/sets/no-comparison,...,None,1062,,No Comparison,245152552,False,2018-12-24T22:00:08Z,2018-12-24T22:00:08Z,5,"549660528,549660318,549659919,549659742,549658101"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3760,https://i1.sndcdn.com/artworks-000247209438-zk...,2017-10-14T19:00:50Z,35217852,,359984842,None,2019-01-14T02:14:46Z,all-rights-reserved,1,https://soundcloud.com/myapplepl/sets/myapple-...,...,None,0,,MyApple Weekly,70584473,False,2017-10-14T19:00:50Z,2017-10-14T19:00:50Z,29,"344973297,345914202,346903456,347914874,349169..."
3761,https://i1.sndcdn.com/artworks-000247209167-pl...,2014-12-09T22:39:17Z,263092108,Technology,63234214,,2017-10-14T18:57:52Z,all-rights-reserved,43,https://soundcloud.com/myapplepl/sets/myapple-...,...,None,2,podcast myapple daily apple iphone ipad mac,MyApple Daily,70584473,False,None,2014-12-09T22:39:17Z,431,"180772908,180913275,181043946,181207702,181649..."
3762,https://i1.sndcdn.com/artworks-aFz7PdQlvKJMLRu...,2013-12-22T20:16:03Z,973451449,Technology,17908634,None,2021-10-31T16:48:29Z,all-rights-reserved,40,https://soundcloud.com/myapplepl/sets/macgadka,...,None,3,apple mac iphone ipad macos ios,MacGadka 🎙,70584473,False,None,2013-12-22T20:16:03Z,138,"126061195,128131427,130886926,132869020,133435..."
3763,None,2013-12-16T10:16:59Z,43472259,,17326482,,2013-12-17T20:49:30Z,all-rights-reserved,6,https://soundcloud.com/myapplepl/sets/podcast-...,...,None,1,,Podcast MyApple 2006 Season 01,70584473,False,None,2013-12-16T10:16:59Z,15,"125005362,125003313,126066543,126069672,126071..."


In [15]:
playlist_df.to_csv('playlist.csv')

Từ các playlist đã thu thập, chọn ra tối đa 5 tracks đầu tiên để lấy dữ liệu

In [16]:
track_raw_data

0       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
1       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
2       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
3       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
4       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
                              ...                        
3760    [{'artwork_url': 'https://i1.sndcdn.com/artwor...
3761    [{'artwork_url': None, 'caption': None, 'comme...
3762    [{'artwork_url': None, 'caption': None, 'comme...
3763    [{'artwork_url': None, 'caption': None, 'comme...
3764    [{'artwork_url': 'https://i1.sndcdn.com/artwor...
Name: tracks, Length: 3765, dtype: object

In [17]:
track_attribute = ['artwork_url', 'commentable', 'comment_count', 'created_at', 'downloadable', 'download_count', 'duration', 
                   'full_duration', 'genre', 'has_downloads_left', 'id', 'last_modified', 'license', 'likes_count', 
                   'permalink_url', 'playback_count', 'purchase_title', 'purchase_url', 'release_date', 'reposts_count', 
                   'state', 'streamable', 'tag_list', 'title', 'user_id', 'visuals', 'waveform_url', 'policy']
track_data = []

for tracks in track_raw_data:
    for track in tracks:
        if 'created_at' in track:
            row = [track[att] for att in track_attribute]
            track_data.append(row)

track_df = pd.DataFrame(data=track_data, columns=track_attribute)
track_df

,artwork_url,commentable,comment_count,created_at,downloadable,download_count,duration,full_duration,genre,has_downloads_left,...,release_date,reposts_count,state,streamable,tag_list,title,user_id,visuals,waveform_url,policy
0,https://i1.sndcdn.com/artworks-et1VIoyP05ZyVZm...,True,84.0,2020-02-18T23:55:03Z,False,0.0,120137,120137,Hip-hop & Rap,False,...,None,125,finished,True,lilcandypaint,Selfish(prod.menohbeats),179256009,None,https://wave.sndcdn.com/ww0SNpBPR5OG_m.json,ALLOW
1,https://i1.sndcdn.com/artworks-C5K3u1isQQKUYto...,True,61.0,2020-02-18T23:55:22Z,False,0.0,133016,133016,Hip-hop & Rap,False,...,None,110,finished,True,lilcandypaint,Angles (Prod. 5heriff X Hippiedren),179256009,None,https://wave.sndcdn.com/gY93cVyBxk3V_m.json,ALLOW
2,https://i1.sndcdn.com/artworks-55eTVQe7bdNJmWn...,True,106.0,2020-02-18T23:55:12Z,False,0.0,115801,115801,Hip-hop & Rap,False,...,None,128,finished,True,lilcandypaint,fishtail (Prod.5heriff),179256009,None,https://wave.sndcdn.com/U8RUkyp9nm5s_m.json,ALLOW
3,https://i1.sndcdn.com/artworks-vfPtpBd89UUC6BA...,True,100.0,2020-02-18T23:55:30Z,False,0.0,104072,104072,Hip-hop & Rap,False,...,None,121,finished,True,lilcandypaint,2018 (prod.litooo),179256009,None,https://wave.sndcdn.com/WDYQhVzSV9nf_m.json,ALLOW
4,https://i1.sndcdn.com/artworks-000434495703-p6...,True,37.0,2018-11-05T23:18:05Z,False,0.0,83520,83520,Hip-hop & Rap,False,...,None,102,finished,True,,lilcandypaint- clouds (prod. Trapboytango) [HO...,179256009,None,https://wave.sndcdn.com/UDHS8TwPaeEe_m.json,ALLOW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14182,https://i1.sndcdn.com/artworks-000165623086-7l...,True,NaN,2016-06-03T00:19:48Z,True,NaN,6455645,6455645,Tecnologia,True,...,None,0,finished,True,"Investimentos CDB LCI LCA ""Tesouro Direto""",042 – Independência Financeira,11458857,None,https://wave.sndcdn.com/HWECTrjoJvdn_m.json,ALLOW
14183,https://i1.sndcdn.com/artworks-000187021607-79...,True,NaN,2016-10-07T00:10:43Z,True,NaN,4981077,4981077,Negócio,True,...,None,0,finished,True,"Podcast Tecnologia Investimentos Robôs ""Inteli...",050 – Investindo com Robôs,11458857,None,https://wave.sndcdn.com/GnaTFqRNen3c_m.json,ALLOW
14184,https://i1.sndcdn.com/artworks-000237806790-5k...,True,NaN,2017-08-11T00:28:14Z,True,NaN,4799012,4799012,Negócios,True,...,None,0,finished,True,Podcast Criptomoeda Blockchain Bitcoin Investi...,071 - Vale a pena investir em bitcoin?,11458857,None,https://wave.sndcdn.com/nIJO9TK2cQT9_m.json,ALLOW
14185,https://i1.sndcdn.com/artworks-000243749595-s2...,True,NaN,2017-09-21T22:34:33Z,True,NaN,5180985,5180985,Negócios,True,...,None,0,finished,True,"Podcast Investimentos Bolsa Ações Ibovespa ""Te...",074 – Investindo com Robôs pt2,11458857,None,https://wave.sndcdn.com/JbEONsZYSoTv_m.json,ALLOW


In [18]:
track_df.to_csv('track.csv')